In [74]:
import os
import pandas as pd

In [75]:
vibration_data = []
machine_status = []

data_list = os.listdir("./data/xep-vibration-data")

for w in data_list:
    if "week" in w:
        week_data_list = os.listdir(f"./data/xep-vibration-data/{w}")
        for d in week_data_list:
            if "vibrationdata" in d:
                vibration_data.append(f"./data/xep-vibration-data/{w}/{d}")
            elif "machinestatus" in d:
                machine_status.append(f"./data/xep-vibration-data/{w}/{d}")

In [76]:
vibration_df = pd.concat([pd.read_csv(v) for v in vibration_data], ignore_index=True)
machine_status_df = pd.concat([pd.read_csv(m) for m in machine_status], ignore_index=True)

In [77]:
vibration_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107316 entries, 0 to 107315
Data columns (total 41 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Timestamp                107275 non-null  float64
 1   SensorID                 107304 non-null  object 
 2   SequenceNumber           107316 non-null  object 
 3   TotalLength              107275 non-null  float64
 4   SourceAddress            0 non-null       float64
 5   TempHumiRange            107275 non-null  float64
 6   TempHumiStatus           107275 non-null  float64
 7   TempHumiEvent            107275 non-null  float64
 8   TempHumiSenVal           107275 non-null  float64
 9   XSenEvent                107275 non-null  float64
 10  XOAVelocity              107275 non-null  float64
 11  XPeakmg                  107275 non-null  float64
 12  XRMSmg                   107275 non-null  float64
 13  XKurtosis                107275 non-null  float64
 14  XCre

In [78]:
machine_status_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332190 entries, 0 to 1332189
Data columns (total 23 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   Timestamp  1332190 non-null  int64  
 1   UID        1332190 non-null  object 
 2   machine1   451137 non-null   float64
 3   machine2   451137 non-null   float64
 4   machine3   451137 non-null   float64
 5   machine4   451137 non-null   float64
 6   machine5   451137 non-null   float64
 7   machine6   452560 non-null   float64
 8   machine7   452560 non-null   float64
 9   machine8   452560 non-null   float64
 10  machine9   452560 non-null   float64
 11  machine10  452560 non-null   float64
 12  machine11  452560 non-null   float64
 13  machine12  452560 non-null   float64
 14  machine13  452560 non-null   float64
 15  machine14  428493 non-null   float64
 16  machine15  428493 non-null   float64
 17  machine16  428493 non-null   float64
 18  machine17  428493 non-null   float64
 19  

In [79]:
vibration_df = vibration_df.dropna(subset=["Timestamp"])
machine_status_df = machine_status_df.dropna(subset=["Timestamp"])

In [80]:
vibration_df["Timestamp"] = vibration_df["Timestamp"].astype("int64")
vibration_df["DeviceTime"] = vibration_df["DeviceTime"].astype("int64")

In [81]:
vibration_df.duplicated(["Timestamp", "SensorID"]).value_counts()

False    107254
True         21
dtype: int64

In [82]:
first = vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="first")
].reset_index(drop=True)

last = vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="last")
].reset_index(drop=True)

first.equals(last)

False

In [83]:
vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="first") |
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="last")
].to_csv("test.csv", index=False)

In [84]:
first = vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="first") &
    vibration_df["Timestamp"] != 1676435835000
].reset_index(drop=True)

last = vibration_df.loc[
    vibration_df.duplicated(["Timestamp", "SensorID"], keep="last") &
    vibration_df["Timestamp"] != 1676435835000
].reset_index(drop=True)

first.equals(last)

True

In [85]:
vibration_df = vibration_df.drop_duplicates(["Timestamp", "SensorID"])

In [86]:
vibration_df.shape

(107254, 41)

In [87]:
machine_status_df.duplicated(["Timestamp", "UID"]).value_counts()

False    1332178
True          12
dtype: int64

In [88]:
first = machine_status_df.loc[
    machine_status_df.duplicated(["Timestamp", "UID"], keep="first")
].reset_index(drop=True)

last = machine_status_df.loc[
    machine_status_df.duplicated(["Timestamp", "UID"], keep="last")
].reset_index(drop=True)

first.equals(last)

True

In [89]:
machine_status_df = machine_status_df.drop_duplicates(["Timestamp", "UID"])

In [90]:
machine_status_df.shape

(1332178, 23)

In [91]:
((vibration_df["Timestamp"] - vibration_df["DeviceTime"]).abs() / (1000 * 60)).describe()

count    1.072540e+05
mean     2.802234e+07
std      1.455832e+05
min      2.778435e+07
25%      2.789311e+07
50%      2.801534e+07
75%      2.814882e+07
max      2.828432e+07
dtype: float64

In [92]:
vibration_df["Timestamp"] = vibration_df["DeviceTime"]

In [93]:
vibration_df = vibration_df.sort_values("Timestamp", ignore_index=True)
machine_status_df = machine_status_df.sort_values("Timestamp", ignore_index=True)

In [94]:
machine_status_df["UID"].value_counts()

sensor2    452556
sensor1    451133
sensor3    428489
Name: UID, dtype: int64

In [95]:
sensor1 = machine_status_df.loc[machine_status_df["UID"] == "sensor1"]
sensor1.isnull().sum()

Timestamp         0
UID               0
machine1          0
machine2          0
machine3          0
machine4          0
machine5          0
machine6     451133
machine7     451133
machine8     451133
machine9     451133
machine10    451133
machine11    451133
machine12    451133
machine13    451133
machine14    451133
machine15    451133
machine16    451133
machine17    451133
machine18    451133
machine19    451133
machine20    451133
machine21    451133
dtype: int64

In [96]:
sensor2 = machine_status_df.loc[machine_status_df["UID"] == "sensor2"]
sensor2.isnull().sum()

Timestamp         0
UID               0
machine1     452556
machine2     452556
machine3     452556
machine4     452556
machine5     452556
machine6          0
machine7          0
machine8          0
machine9          0
machine10         0
machine11         0
machine12         0
machine13         0
machine14    452556
machine15    452556
machine16    452556
machine17    452556
machine18    452556
machine19    452556
machine20    452556
machine21    452556
dtype: int64

In [97]:
sensor3 = machine_status_df.loc[machine_status_df["UID"] == "sensor3"]
sensor3.isnull().sum()

Timestamp         0
UID               0
machine1     428489
machine2     428489
machine3     428489
machine4     428489
machine5     428489
machine6     428489
machine7     428489
machine8     428489
machine9     428489
machine10    428489
machine11    428489
machine12    428489
machine13    428489
machine14         0
machine15         0
machine16         0
machine17         0
machine18         0
machine19         0
machine20         0
machine21         0
dtype: int64

In [98]:
sensor1 = sensor1.dropna(axis=1).drop(columns=["UID"])
sensor2 = sensor2.dropna(axis=1).drop(columns=["UID"])
sensor3 = sensor3.dropna(axis=1).drop(columns=["UID"])

sensor1["sensor1_Timestamp"] = sensor1["Timestamp"]
sensor2["sensor2_Timestamp"] = sensor2["Timestamp"]
sensor3["sensor3_Timestamp"] = sensor3["Timestamp"]

In [99]:
mapped_df = (
    pd.merge_asof(
        pd.merge_asof(
            pd.merge_asof(
                vibration_df, sensor1, on="Timestamp", direction="nearest"
            ),
            sensor2, on="Timestamp", direction="nearest"
        ),
        sensor3, on="Timestamp", direction="nearest"
    )
)

In [100]:
mapped_df["s1_Timestamp_minutes_diff"] = (mapped_df["Timestamp"] - mapped_df["sensor1_Timestamp"]).abs() / (1000 * 60)
mapped_df["s2_Timestamp_minutes_diff"] = (mapped_df["Timestamp"] - mapped_df["sensor2_Timestamp"]).abs() / (1000 * 60)
mapped_df["s3_Timestamp_minutes_diff"] = (mapped_df["Timestamp"] - mapped_df["sensor3_Timestamp"]).abs() / (1000 * 60)

In [101]:
mapped_df[[
    "Timestamp", "s1_Timestamp_minutes_diff", "s2_Timestamp_minutes_diff", "s3_Timestamp_minutes_diff"
]].describe()

,Timestamp,s1_Timestamp_minutes_diff,s2_Timestamp_minutes_diff,s3_Timestamp_minutes_diff
count,1.072540e+05,1.072540e+05,1.072540e+05,1.072540e+05
mean,1.618749e+09,2.778544e+07,2.778544e+07,2.778518e+07
std,5.903688e+07,9.839481e+02,9.839481e+02,9.839481e+02
min,1.552536e+09,2.778431e+07,2.778431e+07,2.778406e+07
25%,1.557700e+09,2.778449e+07,2.778449e+07,2.778423e+07
50%,1.669070e+09,2.778460e+07,2.778460e+07,2.778434e+07
75%,1.675597e+09,2.778645e+07,2.778645e+07,2.778620e+07
max,1.686200e+09,2.778654e+07,2.778654e+07,2.778629e+07


In [102]:
tolerance = 15 / 4
tolerance

3.75

In [103]:
mapped_df.loc[
    (mapped_df["s1_Timestamp_minutes_diff"] > tolerance) |
    (mapped_df["s2_Timestamp_minutes_diff"] > tolerance) |
    (mapped_df["s3_Timestamp_minutes_diff"] > tolerance)
]

,Timestamp,SensorID,SequenceNumber,TotalLength,SourceAddress,TempHumiRange,TempHumiStatus,TempHumiEvent,TempHumiSenVal,XSenEvent,...,machine16,machine17,machine18,machine19,machine20,machine21,sensor3_Timestamp,s1_Timestamp_minutes_diff,s2_Timestamp_minutes_diff,s3_Timestamp_minutes_diff
0,1552536013,sensor-2,3,88.0,NaN,0.0,0.0,0.0,32687.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1668729636518,2.778654e+07,2.778654e+07,2.778629e+07
1,1552536013,sensor-2,3,88.0,NaN,0.0,0.0,0.0,32062.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1668729636518,2.778654e+07,2.778654e+07,2.778629e+07
2,1552536013,sensor-2,3,88.0,NaN,0.0,0.0,0.0,32687.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1668729636518,2.778654e+07,2.778654e+07,2.778629e+07
3,1552536014,sensor-3,3,88.0,NaN,0.0,0.0,0.0,33750.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1668729636518,2.778654e+07,2.778654e+07,2.778629e+07
4,1552536014,sensor-4,3,88.0,NaN,0.0,0.0,0.0,30312.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1668729636518,2.778654e+07,2.778654e+07,2.778629e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107249,1686196238,sensor-1,51,88.0,NaN,0.0,0.0,0.0,30375.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1668729636518,2.778431e+07,2.778431e+07,2.778406e+07
107250,1686197138,sensor-1,52,88.0,NaN,0.0,0.0,0.0,30375.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1668729636518,2.778431e+07,2.778431e+07,2.778406e+07
107251,1686198038,sensor-1,53,88.0,NaN,0.0,0.0,0.0,30500.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1668729636518,2.778431e+07,2.778431e+07,2.778406e+07
107252,1686198938,sensor-1,54,88.0,NaN,0.0,0.0,0.0,30687.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1668729636518,2.778431e+07,2.778431e+07,2.778406e+07


In [104]:
s1_m = [f"machine{i}" for i in range(1, 6)]
s2_m = [f"machine{i}" for i in range(6, 14)]
s3_m = [f"machine{i}" for i in range(14, 22)]

def handle_outlier_machine_status(row):
    if row["s1_Timestamp_minutes_diff"] > tolerance:
        row[s1_m] = 0
    if row["s2_Timestamp_minutes_diff"] > tolerance:
        row[s2_m] = 0
    if row["s3_Timestamp_minutes_diff"] > tolerance:
        row[s3_m] = 0

    return row

In [105]:
mapped_df = mapped_df.apply(
    handle_outlier_machine_status,
    axis=1,
)

In [106]:
mapped_df = mapped_df.drop(columns=[
    "sensor1_Timestamp", "sensor2_Timestamp", "sensor3_Timestamp",
    "s1_Timestamp_minutes_diff", "s2_Timestamp_minutes_diff", "s3_Timestamp_minutes_diff",
])

In [107]:
mapped_df["DateTime"] = pd.to_datetime(mapped_df["Timestamp"], unit="ms")
mapped_df["DateTime"] = mapped_df["DateTime"].dt.strftime("%Y-%m-%d %H:%M:%S")

front_cols = ["Timestamp", "DateTime"]
mapped_df = mapped_df[front_cols + [col for col in mapped_df if col not in front_cols]]

mapped_df = mapped_df.sort_values("Timestamp", ignore_index=True)

In [108]:
mapped_df

,Timestamp,DateTime,SensorID,SequenceNumber,TotalLength,SourceAddress,TempHumiRange,TempHumiStatus,TempHumiEvent,TempHumiSenVal,...,machine12,machine13,machine14,machine15,machine16,machine17,machine18,machine19,machine20,machine21
0,1552536013,1970-01-18 23:15:36,sensor-2,3,88.0,NaN,0.0,0.0,0.0,32687.0,...,0,0,0,0,0,0,0,0,0,0
1,1552536013,1970-01-18 23:15:36,sensor-2,3,88.0,NaN,0.0,0.0,0.0,32062.0,...,0,0,0,0,0,0,0,0,0,0
2,1552536013,1970-01-18 23:15:36,sensor-2,3,88.0,NaN,0.0,0.0,0.0,32687.0,...,0,0,0,0,0,0,0,0,0,0
3,1552536014,1970-01-18 23:15:36,sensor-3,3,88.0,NaN,0.0,0.0,0.0,33750.0,...,0,0,0,0,0,0,0,0,0,0
4,1552536014,1970-01-18 23:15:36,sensor-4,3,88.0,NaN,0.0,0.0,0.0,30312.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107249,1686196238,1970-01-20 12:23:16,sensor-1,51,88.0,NaN,0.0,0.0,0.0,30375.0,...,0,0,0,0,0,0,0,0,0,0
107250,1686197138,1970-01-20 12:23:17,sensor-1,52,88.0,NaN,0.0,0.0,0.0,30375.0,...,0,0,0,0,0,0,0,0,0,0
107251,1686198038,1970-01-20 12:23:18,sensor-1,53,88.0,NaN,0.0,0.0,0.0,30500.0,...,0,0,0,0,0,0,0,0,0,0
107252,1686198938,1970-01-20 12:23:18,sensor-1,54,88.0,NaN,0.0,0.0,0.0,30687.0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
mapped_df.duplicated(["Timestamp"]).value_counts()

False    96239
True     11015
dtype: int64

In [110]:
mapped_df[(1669578553000 <= mapped_df["Timestamp"]) & (mapped_df["Timestamp"] < 1669578554000)]

,Timestamp,DateTime,SensorID,SequenceNumber,TotalLength,SourceAddress,TempHumiRange,TempHumiStatus,TempHumiEvent,TempHumiSenVal,...,machine12,machine13,machine14,machine15,machine16,machine17,machine18,machine19,machine20,machine21


In [111]:
vibration_df.to_csv("./data/vibrationdata.csv", index=False)
machine_status_df.to_csv("./data/machinestatus.csv", index=False)
mapped_df.to_csv("./data/mapped.csv", index=False)